In [9]:
import numpy as np
import cv2
import imageio.v3 as iio
import matplotlib.pyplot as plt
from typing import Tuple


In [10]:
from horn_schunk_estimator import horn_schunk_estimator
from liu_shen_estimator import liu_shen_estimator

In [11]:

def correction_illumination(I1: np.ndarray, I2: np.ndarray, window_shifting: np.ndarray, size_average: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Adjust the illumination of images I1 and I2 based on a window shifting and averaging process.

    Args:
        I1 (np.ndarray): First input image.
        I2 (np.ndarray): Second input image.
        window_shifting (np.ndarray): Window shifting parameters.
        size_average (int): Size for averaging.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Adjusted images I1 and I2.
    """
    I1 = I1.astype(np.float64)
    I2 = I2.astype(np.float64)

    x3, y3, x4, y4 = window_shifting[:4]
    I1_mean = np.mean(I1[y3:y4, x3:x4])
    I2_mean = np.mean(I2[y3:y4, x3:x4])
    R12 = I1_mean / I2_mean
    I2 = R12 * I2

    if size_average > 0:
        N = size_average
        h = np.ones((N, N)) / (N * N)
        I12F = cv2.filter2D(I1, -1, h) - cv2.filter2D(I2, -1, h)
        I2 = I2 + I12F
    else:
        I2 = I2

    return I1, I2


In [12]:
from skimage.filters import gaussian
from skimage.transform import rescale


def pre_processing_a(I1: np.ndarray, I2: np.ndarray, scale_im: float, size_filter: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Pre-process images to reduce random noise and downsample if displacements are large.

    Args:
        I1 (np.ndarray): First input image.
        I2 (np.ndarray): Second input image.
        scale_im (float): Scale factor for downsizing images.
        size_filter (int): Size of the Gaussian filter for removing random noise.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Pre-processed images I1 and I2.
    """
    I1 = gaussian(I1, sigma=size_filter)
    I2 = gaussian(I2, sigma=size_filter)

    if scale_im < 1:
        I1 = rescale(I1, scale_im, anti_aliasing=True)
        I2 = rescale(I2, scale_im, anti_aliasing=True)

    return I1, I2


In [13]:
from scipy.ndimage import convolve
from skimage.filters import laplace

def OpticalFlowPhysics_fun(I1, I2, lambda_1, lambda_2):
    D1 = np.array([[0, 0, 0], [0, -1, -1], [0, 1, 1]]) / 2
    F1 = np.array([[0, 0, 0], [0, 1, 1], [0, 1, 1]]) / 4

    Ix = convolve((I1 + I2) / 2, D1, mode='reflect')
    Iy = convolve((I1 + I2) / 2, D1.T, mode='reflect')
    It = convolve(I2 - I1, F1, mode='reflect')

    maxnum_1 = 500
    tol_1 = 1e-12

    u, v = horn_schunk_estimator(Ix, Iy, It, lambda_1, tol_1, maxnum_1)
    ux_horn = v
    uy_horn = u

    Dm = 0 * 1e-3
    f = Dm * laplace(I1)

    maxnum = 60
    tol = 1e-8

    dx = 1
    dt = 1

    u, v, error1 = liu_shen_estimator(I1, I2, f, dx, dt, lambda_2, tol, maxnum, uy_horn, ux_horn)

    ux = v
    uy = u

    vor = vorticity(ux, uy)

    return ux, uy, vor, ux_horn, uy_horn, error1

In [14]:

def shift_image_fun_refine_1(ux_corr: np.ndarray, uy_corr: np.ndarray, Im1: np.ndarray, Im2: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Shift the image Im1 based on the velocity field (ux_corr, uy_corr) and compute the velocity difference for iterative correction.

    Args:
        ux_corr (np.ndarray): Velocity field in the x-direction.
        uy_corr (np.ndarray): Velocity field in the y-direction.
        Im1 (np.ndarray): First input image.
        Im2 (np.ndarray): Second input image.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray]: Shifted image Im1, and velocity differences uxI, uyI.
    """
    rows, cols = Im1.shape
    flow = np.stack((ux_corr, uy_corr), axis=-1)
    Im1_shift = cv2.remap(Im1, flow, None, cv2.INTER_LINEAR)

    flow_inv = np.stack((-ux_corr, -uy_corr), axis=-1)
    Im2_shift = cv2.remap(Im2, flow_inv, None, cv2.INTER_LINEAR)

    uxI, uyI = cv2.calcOpticalFlowFarneback(Im1_shift, Im2_shift, None, 0.5, 3, 15, 3, 5, 1.2, 0)[..., 0], cv2.calcOpticalFlowFarneback(Im1_shift, Im2_shift, None, 0.5, 3, 15, 3, 5, 1.2, 0)[..., 1]

    return Im1_shift, uxI, uyI


In [15]:
"""
Main program for extraction of velocity field from a pair of flow visualization images.
"""
# Read a pair of images
Im1 = iio.imread('../images/White_Oval_1.tif')
Im2 = iio.imread('../images/White_Oval_2.tif')

# Set the parameters for optical flow computation
lambda_1 = 20  # Horn-Schunck estimator for initial field
lambda_2 = 2000  # Liu-Shen estimator for refined estimation
no_iteration = 1  # Number of iterations in the coarse-to-fine iterative process
scale_im = 0.5  # Scale factor for downsizing images
size_average = 0  # Size for averaging to bypass local illumination intensity adjustment
size_filter = 4  # Gaussian filter size for removing random noise
index_region = 0  # Select a region for processing (1) or process the whole image (0)

# Select a region of interest for diagnostics
# if index_region == 1:
#     plt.imshow(Im1, cmap='gray')
#     plt.colorbar()
#     plt.axis('image')
#     plt.show()

#     xy = plt.ginput(2)
#     x1, x2 = int(np.floor(min(xy[:, 0]))), int(np.floor(max(xy[:, 0])))
#     y1, y2 = int(np.floor(min(xy[:, 1]))), int(np.floor(max(xy[:, 1])))
#     Im1 = Im1[y1:y2, x1:x2]
#     Im2 = Im2[y1:y2, x1:x2]
# elif index_region == 0:
#     pass

Im1_original = Im1.copy()
Im2_original = Im2.copy()

# Correct the global and local intensity change in images
window_shifting = np.array([1, Im1.shape[0], 1, Im1.shape[1]])
Im1, Im2 = correction_illumination(Im1, Im2, window_shifting, size_average)

# Pre-process for reducing random noise and downsampling images if displacements are large
Im1, Im2 = pre_processing_a(Im1, Im2, scale_im, size_filter)

I_region1 = Im1.copy()
I_region2 = Im2.copy()

# Initial optical flow calculation for a coarse-grained velocity field
ux0, uy0, vor, ux_horn, uy_horn, error1 = optical_flow_physics_fun(I_region1, I_region2, lambda_1, lambda_2)

# Generate the shifted image from Im1 based on the initial coarse-grained velocity field (ux0, uy0)
Im1 = cv2.convertScaleAbs(Im1_original)
Im2 = cv2.convertScaleAbs(Im2_original)

ux_corr = ux0.copy()
uy_corr = uy0.copy()

# Estimate the displacement vector and make correction in iterations
k = 1
while k <= no_iteration:
    Im1_shift, uxI, uyI = shift_image_fun_refine_1(ux_corr, uy_corr, Im1, Im2)

    # Calculation of correction of the optical flow
    ux_corr, uy_corr, vor, ux_horn, uy_horn, error2 = OpticalFlowPhysics_fun(Im1_shift, Im2, lambda_1, lambda_2)

    k += 1

# Refined velocity field
ux = ux_corr
uy = uy_corr

# Show the images and processed results
plt.figure()
plt.subplot(2, 2, 1)
plt.imshow(Im1_original, cmap='gray')
plt.title('Original Image 1')
plt.colorbar()

plt.subplot(2, 2, 2)
plt.imshow(Im2_original, cmap='gray')
plt.title('Original Image 2')
plt.colorbar()

plt.subplot(2, 2, 3)
plt.quiver(ux, uy, scale=1, angles='xy', scale_units='xy')
plt.title('Velocity Field')
plt.axis('equal')

plt.subplot(2, 2, 4)
plt.imshow(vor, cmap='jet')
plt.title('Velocity Magnitude')
plt.colorbar()

plt.show()


/home/user/miniconda3/envs/openpiv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/user/miniconda3/envs/openpiv/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/user/miniconda3/envs/openpiv/lib/python3.12/site-packages/skimage/transform/_warps.py:738: RuntimeWarning: All-NaN slice encountered
  min_val = min_func(input_image)
/home/user/miniconda3/envs/openpiv/lib/python3.12/site-packages/skimage/transform/_warps.py:742: RuntimeWarning: All-NaN slice encountered
  max_val = max_func(input_image)


ValueError: operands could not be broadcast together with shapes (119,167) (238,334) 